In [72]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [73]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
#file_beg = '../NHANES-Downloader/data/csv_data/'
file_beg = '../NHANES-Downloader/data/csv_data/'

In [74]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')
files10 = glob.glob(file_beg+'2017-2018/*/*.csv')

In [75]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9, files10]

In [76]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [77]:
bpq_indx = [63, 79, 93, 91, 90, 97, 100, 128, 98, 115]

In [78]:
dfs = defaultdict(int)
for i in range(0,len(bpq_indx)):
    dfs[i] = pd.read_csv(file_list_dict[i][bpq_indx[i]])

## Important Variables

In [79]:
#1999-2018
descr = ['SEQN', 'High Blood Pressure (Y/N)']
var_names = ["SEQN", "BPQ020"]

In [80]:
#To display all columns in Jupyter Notebooks
#pd.set_option('display.max_columns', 500)

## Functions: Recategorize values, Count Values, Drop Rows

In [81]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Make a copy of the dataframes

In [82]:
#1999-2004
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names].copy() #2003-2004
df3 = dfs[3][var_names].copy() #2005-2006
df4 = dfs[4][var_names].copy() #2007-2008
df5 = dfs[5][var_names].copy() #2009-2010
df6 = dfs[6][var_names].copy() #2011-2012
df7 = dfs[7][var_names].copy() #2013-2014
df8 = dfs[8][var_names].copy() #2015-2016
df9 = dfs[9][var_names].copy() #2017-2018


## Recategorize values

In [83]:
#No recategorizing needed

## Rename columns 1999 - 2004

In [84]:
#No renaming needed

## Append years 1999 - 2016

In [85]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016","2017-2018"]

In [86]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8, df9]

In [87]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [88]:
result_1999_2018 = pd.concat(frames, keys = years)

In [89]:
result_1999_2018_cleaned = result_1999_2018.copy()

In [90]:
result_1999_2018

SEQN  BPQ020       Year
1999-2000 0          2.0     2.0  1999-2000
          1          5.0     1.0  1999-2000
          2          6.0     2.0  1999-2000
          3          7.0     1.0  1999-2000
          4         10.0     2.0  1999-2000
...                  ...     ...        ...
2017-2018 6156  102950.0     1.0  2017-2018
          6157  102952.0     2.0  2017-2018
          6158  102953.0     2.0  2017-2018
          6159  102954.0     2.0  2017-2018
          6160  102956.0     1.0  2017-2018

[63592 rows x 3 columns]

## Count values for each column

In [91]:
col_names = var_names

In [92]:
len(col_names)

2

In [93]:
#BPQ020 - Ever told you had high blood pressure
count_vals(result_1999_2018, col_names[1])

BPQ020
1.0    19434
2.0    43785
9.0      104
Name: SEQN, dtype: int64 

 NaN:  269


In [94]:
len(result_1999_2018)

63592

## Remove missing values:

In [95]:
drop_rows(result_1999_2018_cleaned, col_names[1], [np.nan, 7, 9])

## See if missing values have been correctly removed:

In [96]:
count_vals(result_1999_2018_cleaned, col_names[1])

BPQ020
1.0    19434
2.0    43785
Name: SEQN, dtype: int64 

 NaN:  0


In [97]:
before = len(result_1999_2018)
before

63592

In [98]:
after = len(result_1999_2018_cleaned)
after

63219

In [99]:
(before-after)/after

0.0059001249624321805

## MongoDB Insertion

In [100]:
#Import MongoClient
from pymongo import MongoClient

#Create a MongoClient to run the MongoDB instance
client = MongoClient('localhost', 27017)

In [101]:
#Connect to existing database
db = client.NHANES
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'NHANES')

In [102]:
#Create collection in database
bpq = db.bpq

In [103]:
#If collections exist, then drop
if 'bpq' in db.list_collection_names():
    bpq.drop()
    db.list_collection_names()

## Create new collection to input into database

In [104]:
result_1999_2018_cleaned[:3]

SEQN  BPQ020       Year
1999-2000 0   2.0     2.0  1999-2000
          1   5.0     1.0  1999-2000
          2   6.0     2.0  1999-2000

In [105]:
#Set SEQN as _id (Primary Key)
result_1999_2018_cleaned.rename(columns={'SEQN':'_id'}, inplace=True)

In [106]:
#Dataframe to dictionary
bpq_dict = result_1999_2018_cleaned.to_dict(orient='records')

In [107]:
bpq_dict[0]

{'_id': 2.0, 'BPQ020': 2.0, 'Year': '1999-2000'}

In [108]:
#Insert collection
bpq.insert_many(bpq_dict)

In [109]:
#View collections
db.list_collection_names()

['alq', 'bpq', 'bmx']